In [2]:
# Import libraries
import numpy as np
import pandas as pd

import os

from scipy.sparse import csr_matrix, csc_matrix
import scipy.sparse as sparse

from scipy.sparse.linalg import svds
from scipy.linalg import sqrtm

import joblib

#ratings = pd.read_csv('/home/quentin/Documents/simplon/20210406_moteur_de_recommandation_de_livre/source/goodbooks-10k-master/ratings.csv', index_col=False)

In [ ]:
n_users = ratings.user_id.unique().shape[0]
n_book = ratings.book_id.unique().shape[0]
print ('Number of users = ' + str(n_users) + ' | Number of books = ' + str(n_book))

print(n_users, n_book)

In [ ]:
id_users = ratings.user_id.unique()
id_book = ratings.book_id.unique()
print ('id of users = ' + str(n_users) + ' | id of books = ' + str(n_book))

In [ ]:
#Ratings = ratings.pivot(index = 'user_id', columns ='book_id', values = 'rating').fillna(0)
#Ratings.head()

In [3]:
Ratings = sparse.load_npz('/home/quentin/Documents/simplon/20210406_moteur_de_recommandation_de_livre/training/sparse_matrix.npz')
print(type(Ratings))

<class 'scipy.sparse.csr.csr_matrix'>


In [4]:
print(Ratings)

  (0, 3)	5.0
  (0, 9)	4.0
  (0, 10)	5.0
  (0, 12)	4.0
  (0, 15)	3.0
  (0, 21)	3.0
  (0, 30)	4.0
  (0, 31)	4.0
  (0, 32)	4.0
  (0, 34)	5.0
  (0, 35)	4.0
  (0, 37)	2.0
  (0, 39)	2.0
  (0, 41)	3.0
  (0, 42)	4.0
  (0, 44)	5.0
  (0, 45)	4.0
  (0, 46)	3.0
  (0, 53)	3.0
  (0, 56)	3.0
  (0, 59)	3.0
  (0, 65)	4.0
  (0, 66)	3.0
  (0, 69)	5.0
  (0, 71)	3.0
  :	:
  (53423, 3564)	4.0
  (53423, 3601)	5.0
  (53423, 3638)	4.0
  (53423, 3714)	4.0
  (53423, 4002)	4.0
  (53423, 4195)	4.0
  (53423, 4213)	5.0
  (53423, 4276)	4.0
  (53423, 4482)	5.0
  (53423, 4644)	5.0
  (53423, 5300)	5.0
  (53423, 5499)	4.0
  (53423, 5716)	5.0
  (53423, 5987)	4.0
  (53423, 6454)	4.0
  (53423, 7211)	4.0
  (53423, 7502)	4.0
  (53423, 7832)	4.0
  (53423, 8212)	4.0
  (53423, 8261)	4.0
  (53423, 8608)	4.0
  (53423, 8725)	4.0
  (53423, 8968)	5.0
  (53423, 9254)	4.0
  (53423, 9848)	4.0


In [ ]:
Ratings.shape

In [ ]:
(np.prod(Ratings.shape)*8)/1000000000

In [4]:
user_ratings_mean = np.mean(Ratings, axis = 1)
user_ratings_mean_r = user_ratings_mean.reshape(-1, 1)
user_ratings_mean_r = csr_matrix(user_ratings_mean_r)

In [6]:
user_ratings_mean_r.shape

(53424, 1)

In [7]:
Ratings.shape

(53424, 10000)

In [19]:
Ratings.todense()

matrix([[0., 0., 0., ..., 0., 0., 0.],
        [0., 5., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [4., 5., 0., ..., 0., 0., 0.],
        [4., 5., 0., ..., 0., 0., 0.],
        [4., 5., 4., ..., 0., 0., 0.]])

In [5]:
import gc

Ratings_demeaned = Ratings - user_ratings_mean_r

gc.collect()

ValueError: inconsistent shapes

In [7]:
type(Ratings_demeaned)

numpy.matrix

In [ ]:
#sparsity = round(1.0 - len(ratings) / float(R.shape[0] * R.shape[1]), 3)
#print ('The sparsity level of book rating dataset is ' +  str(sparsity * 100) + '%')

In [ ]:
U, sigma, Vt = svds(Ratings_demeaned, k = 30)

sigma = np.diag(sigma)

all_user_predicted_ratings = np.dot(np.dot(U, sigma), Vt)# + user_ratings_mean.reshape(-1, 1)

In [ ]:
all_user_predicted_ratings =np.asmatrix(all_user_predicted_ratings) 


In [ ]:
all_user_predicted_ratings = all_user_predicted_ratings + user_ratings_mean_r

In [ ]:
print('ok avant psarce matrix')
sparse_matrix = csr_matrix(all_user_predicted_ratings)
# save csr_matrix
print('ok sparce matrix')

save_path = 'training/'
file_name = "all_user_predicted_ratings.npz"

complete_name = os.path.join(save_path, file_name)

scipy.sparse.save_npz(complete_name, sparse_matrix)

In [ ]:
preds = pd.DataFrame(all_user_predicted_ratings, columns = id_book)
preds.head()